In [2]:
from __future__ import print_function

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import math as m
from time import time
from scipy import stats
import scipy as sp

In [24]:
def N(x):
    return stats.norm.cdf(x, 0.0, 1.0)

def NPrime(x):
    return stats.norm.pdf(x, 0.0, 1.0)

def bsm_d1(S, K, T, r, sigma):
    return (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * m.sqrt(T))

def bsm_d2(S, K, T, r, sigma):
    return (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * m.sqrt(T))

def bsm_pv(isCall, S, K, T, r, sigma):
    d1 = bsm_d1(S, K, T, r, sigma)
    d2 = bsm_d2(S, K, T, r, sigma)
    if isCall:
        return S * N(d1) - K * m.exp(-r * T) * N(d2)
    else:
        return K * N(-d2) * m.exp(-r * T)  - S * N(-d1)

def bsm_delta(isCall, S, K, T, r, sigma):
    d1 = bsm_d1(S, K, T, r, sigma)
    if isCall:
        return N(d1)
    else:
        return -N(-d1)
    
def bsm_gamma(isCall, S, K, T, r, sigma):
    d2 = bsm_d1(S, K, T, r, sigma)
    return NPrime(d2) / (S * sigma * m.sqrt(T))
    
def bsm_vega(isCall, S, K, T, r, sigma):
    d1 = bsm_d1(S, K, T, r, sigma)
    return S * NPrime(d1) * m.sqrt(T)

In [8]:
S0 = 80.; K = 85.; T = 1.0; r = 0.05;
sigma = 0.2
# for euro call
ref_pv = bsm_pv(True, S0, K, T, r, sigma)
ref_delta = bsm_delta(True, S0, K, T, r, sigma)
print( "ref_pv: %.6f, ref_delta: %.6f " % (ref_pv, ref_delta) )

ref_pv: 5.988244, ref_delta: 0.518694 


$\frac{dS}{S} = r dt + \sigma dW_t$


$S(t) = S(t-\Delta t) \exp(r t - \frac{1}{2} \sigma^2 t + \sigma W_t )$

In [9]:
def standard_normal(I):
    z = np.random.standard_normal(I)
    mean = np.mean(z)
    std = np.std(z)
    return (z - mean)/std    

def euro_opt_pv_mc(isCall, S0, K, T, r, sigma, I):
    # Simulating I values
    z = standard_normal(I)
    S = S0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * m.sqrt(T) * z)

    # payoff
    if isCall:
        P = np.maximum(S - K, 0)
    else:
        P = np.maximum(K - S, 0)
    # PV as expected discounted payoff
    C = np.sum( m.exp(-r * T) * P ) / I
    return C, S

# Parameter
I = 50000

np.random.seed(12345)
t0 = time()
C, _ = euro_opt_pv_mc(True, S0, K, T, r, sigma, I)
calcTime = time() - t0

print( "PV: %.5f, abs diff: %.5f, rel diff:  %.5f" % (C, ref_pv - C, (ref_pv - C)/C) )
print( "Calculation time   %.5f" % calcTime )

PV: 5.98935, abs diff: -0.00110, rel diff:  -0.00018
Calculation time   0.00801


$\frac{ \partial C  }{ \partial S} \approx \frac{ C_{i+1} - C_{i-1} }{ 2 \Delta S }$

In [13]:
def euro_opt_delta_mc(isCall, S0, K, T, r, sigma, I, dS):
    C_1, _ = euro_opt_pv_mc(isCall, S0-dS, K, T, r, sigma, I)
    C1, _ = euro_opt_pv_mc(isCall, S0+dS, K, T, r, sigma, I)
    return (C1 - C_1) / (2. * dS)

np.random.seed(12345)
t0 = time()
delta = euro_opt_delta_mc(True, S0, K, T, r, sigma, I, S0/100.)
calcTime = time() - t0

print( "Delta: %.5f, abs diff: %.5f, rel diff:  %.5f" % (delta, ref_delta - delta, (ref_delta - delta)/delta) )
print( "Calculation time   %.5f" % calcTime )

Delta: 0.51888, abs diff: -0.00018, rel diff:  -0.00035
Calculation time   0.01601


----------

## Find spot value for given value of delta ##
### delta neutral position ###

In [31]:
def find_strike_for_delta_a(isCall, S, T, r, sigma, target_delta):
    # >>>
    # modify this
    K_ = S # usesp.optimize.brentq and bsm_delta
    # <<<
    print( "Strike value %.8f" % K_ )
    print( "Call delta: %.8f, gamma %.8f" % (
            bsm_delta(True, S=S, K=K_, T=T, r=r, sigma=sigma),
            bsm_gamma(True, S=S, K=K_, T=T, r=r, sigma=sigma)
        ) )
    print( "Put delta: %.8f, gamma %.8f" % (
            bsm_delta(False, S=S, K=K_, T=T, r=r, sigma=sigma),
            bsm_gamma(False, S=S, K=K_, T=T, r=r, sigma=sigma) 
        ) )

In [32]:
S0 = 80.; T = 1.0; r = 0.05; sigma = 0.2

find_strike_for_delta_a(True, S=S0, T=T, r=r, sigma=sigma, target_delta=0.5)

Strike value 80.00000000
Call delta: 0.63683065, gamma 0.02345252
Put delta: -0.36316935, gamma 0.02345252


In [38]:
def find_strike_for_delta_mc(isCall, S, T, r, sigma, target_delta, I, dS):
    # >>>
    # modify this
    K_ = S # usesp.optimize.brentq and euro_opt_delta_mc
    # <<<
    print( "Strike value %.8f" % K_ )
    print( "Call delta: %.8f, ref_delta %.8f" % (
            euro_opt_delta_mc(True, S0=S, K=K_, T=T, r=r, sigma=sigma, I=I, dS=dS),
            bsm_delta(True, S=S, K=K_, T=T, r=r, sigma=sigma)
        ) )
    print( "Put delta: %.8f, ref_delta %.8f" % (
            euro_opt_delta_mc(False, S0=S, K=K_, T=T, r=r, sigma=sigma, I=I, dS=dS),
            bsm_delta(False, S=S, K=K_, T=T, r=r, sigma=sigma) 
        ) )

In [39]:
np.random.seed(12345)
find_strike_for_delta_mc(True, S=S0, T=T, r=r, sigma=sigma, target_delta=0.5, I=I, dS=S0/100.)

Strike value 80.00000000
Call delta: 0.62797434, ref_delta 0.63683065
Put delta: -0.36196806, ref_delta -0.36316935
